In [1]:
from openmmtools.states import SamplerState, ThermodynamicState, CompoundThermodynamicState
from simtk import unit, openmm
from perses.tests.utils import compute_potential_components
from openmmtools.constants import kB
from openmmtools import cache, utils
from perses.dispersed import feptasks
from perses.dispersed.feptasks import minimize
from perses.dispersed.utils import configure_platform
from perses.annihilation.rest import RESTTopologyFactory
from perses.annihilation.lambda_protocol import RESTState
import numpy as np
from perses.tests.test_topology_proposal import generate_atp, generate_dipeptide_top_pos_sys
from openmmtools.testsystems import AlanineDipeptideVacuum, AlanineDipeptideExplicit
import itertools

cache.global_context_cache.platform = configure_platform(utils.get_fastest_platform().getName())


INFO:rdkit:Enabling RDKit 2020.09.1 jupyter extensions


conducting subsequent work with the following platform: CUDA


conducting subsequent work with the following platform: CUDA


In [2]:
temperature = 300.0 * unit.kelvin
kT = kB * temperature
beta = 1.0/kT
REFERENCE_PLATFORM = openmm.Platform.getPlatformByName("CPU")

In [3]:
# Create vanilla system for alanine dipeptide in solvent
ala_solvent = AlanineDipeptideExplicit()
ala_solvent.system.removeForce(4)
# ala_solvent.system.getForce(3).setNonbondedMethod(openmm.NonbondedForce.NoCutoff)


In [4]:
res1 = list(ala_solvent.topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]

In [8]:
# Set temperatures
T_min = 298.0 * unit.kelvin
T = 600 * unit.kelvin
T = 298.0 * unit.kelvin

# Set sys, pos, top
system = ala_solvent.system
positions = ala_solvent.positions
topology = ala_solvent.topology

In [9]:
class REST2(RESTTopologyFactory):
    """
    subclass REST, but move solvent-solvent to custom nonbonded force
    """
    def _add_nonbonded_force_terms(self):
        from openmmtools.constants import ONE_4PI_EPS0 # OpenMM constant for Coulomb interactions (implicitly in md_unit_system units)

        standard_nonbonded_force = openmm.NonbondedForce()
        custom_nonbonded_expression = f"(4*epsilon*((sigma/r)^12-(sigma/r)^6) + ONE_4PI_EPS0*chargeProd/r) * scale_factor; \
                                        sigma=0.5*(sigma1+sigma2); \
                                        epsilon=sqrt(epsilon1*epsilon2); \
                                        ONE_4PI_EPS0 = {ONE_4PI_EPS0}; \
                                        chargeProd=q1*q2;"

        custom_nonbonded_expression += self.scaling_expression(nb=True)
        custom_nonbonded_force = openmm.CustomNonbondedForce(custom_nonbonded_expression)

#         self._out_system.addForce(standard_nonbonded_force)
#         self._out_system_forces[standard_nonbonded_force.__class__.__name__] = standard_nonbonded_force

        self._out_system.addForce(custom_nonbonded_force)
        self._out_system_forces[custom_nonbonded_force.__class__.__name__] = custom_nonbonded_force

        #set the appropriate parameters
        epsilon_solvent = self._og_system_forces['NonbondedForce'].getReactionFieldDielectric()
        r_cutoff = self._og_system_forces['NonbondedForce'].getCutoffDistance()
        if self._nonbonded_method != openmm.NonbondedForce.NoCutoff:
            standard_nonbonded_force.setReactionFieldDielectric(epsilon_solvent)
            standard_nonbonded_force.setCutoffDistance(r_cutoff)
            custom_nonbonded_force.setCutoffDistance(r_cutoff)
        if self._nonbonded_method in [openmm.NonbondedForce.PME, openmm.NonbondedForce.Ewald]:
            [alpha_ewald, nx, ny, nz] = self._og_system_forces['NonbondedForce'].getPMEParameters()
            delta = self._og_system_forces['NonbondedForce'].getEwaldErrorTolerance()
            standard_nonbonded_force.setPMEParameters(alpha_ewald, nx, ny, nz)
            standard_nonbonded_force.setEwaldErrorTolerance(delta)
        standard_nonbonded_force.setNonbondedMethod(self._nonbonded_method)
        custom_nonbonded_force.setNonbondedMethod(self._translate_nonbonded_method_to_custom(self._nonbonded_method))

        #translate nonbonded to custom
        if self._og_system_forces['NonbondedForce'].getUseDispersionCorrection():
#             self._out_system_forces['NonbondedForce'].setUseDispersionCorrection(True)
            if self._use_dispersion_correction:
                custom_nonbonded_force.setUseLongRangeCorrection(True)
        else:
            custom_nonbonded_force.setUseLongRangeCorrection(False)

        if self._og_system_forces['NonbondedForce'].getUseSwitchingFunction():
            switching_distance = self._og_system_forces['NonbondedForce'].getSwitchingDistance()
            standard_nonbonded_force.setUseSwitchingFunction(True)
            standard_nonbonded_force.setSwitchingDistance(switching_distance)
            custom_nonbonded_force.setUseSwitchingFunction(True)
            custom_nonbonded_force.setSwitchingDistance(switching_distance)
        else:
            standard_nonbonded_force.setUseSwitchingFunction(False)
            custom_nonbonded_force.setUseSwitchingFunction(False)

        custom_nonbonded_force.addPerParticleParameter("q")
        custom_nonbonded_force.addPerParticleParameter("sigma")
        custom_nonbonded_force.addPerParticleParameter("epsilon")
        custom_nonbonded_force.addPerParticleParameter("identifier")

        custom_nonbonded_force.addGlobalParameter('solute_scale', 1.0)
        custom_nonbonded_force.addGlobalParameter('inter_scale', 1.0)

        #finally, make a custombondedforce to treat the exceptions
        custom_bonded_expression = f"(4*epsilon*((sigma/r)^12-(sigma/r)^6) + ONE_4PI_EPS0*chargeProd/r) * scale_factor; \
                                        ONE_4PI_EPS0 = {ONE_4PI_EPS0};"

        custom_bonded_expression += self.scaling_expression()

        custom_bond_force = openmm.CustomBondForce(custom_bonded_expression)
        self._out_system.addForce(custom_bond_force)
        self._out_system_forces["CustomExceptionForce"] = custom_bond_force

        #charges
        custom_bond_force.addPerBondParameter("chargeProd")

        #sigma
        custom_bond_force.addPerBondParameter("sigma")

        #epsilon
        custom_bond_force.addPerBondParameter("epsilon")

        #identifier
        custom_bond_force.addPerBondParameter("identifier")

        #global params
        custom_bond_force.addGlobalParameter('solute_scale', 1.0)
        custom_bond_force.addGlobalParameter('inter_scale', 1.0)
    
    def _add_nonbondeds(self):
        self._solute_exceptions, self._interexceptions, self._solvent_exceptions = [], [], []

        #the output nonbonded force _only_ contains solvent atoms (the rest are zeroed); same with exceptions
        """
        First, handle the NonbondedForce in the out_system
        """
        og_nb_force = self._og_system_forces['NonbondedForce']
        for particle_idx in range(self._num_particles):
            q, sigma, epsilon = og_nb_force.getParticleParameters(particle_idx)
            identifier = self.get_identifier(particle_idx)

            if identifier == 1:
#                 self._out_system_forces['NonbondedForce'].addParticle(q, sigma, epsilon)
                self._out_system_forces['CustomNonbondedForce'].addParticle([q, sigma, epsilon, identifier])
            else:
#                 self._out_system_forces['NonbondedForce'].addParticle(q*0.0, sigma, epsilon*0.0)
                self._out_system_forces['CustomNonbondedForce'].addParticle([q, sigma, epsilon, identifier])

        #add appropriate interaction group
        solute_ig, solvent_ig = set(self._solute_region), set(self._solvent_region)
        self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solute_ig, solvent_ig)
        self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solute_ig, solute_ig)
        self._out_system_forces['CustomNonbondedForce'].addInteractionGroup(solvent_ig, solvent_ig)

        #handle exceptions
        for exception_idx in range(og_nb_force.getNumExceptions()):
            p1, p2, chargeProd, sigma, epsilon = og_nb_force.getExceptionParameters(exception_idx)
            identifier = self.get_identifier([p1, p2])
            if identifier == 1:
#                 self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd, sigma, epsilon)
                self._solvent_exceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions
            elif identifier == 0:
                self._solute_exceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
#                 self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions
            elif identifier == 2:
                self._interexceptions.append([p1, p2, [chargeProd, sigma, epsilon]])
#                 self._out_system_forces['NonbondedForce'].addException(p1, p2, chargeProd*0.0, sigma, epsilon*0.0)
                self._out_system_forces['CustomNonbondedForce'].addExclusion(p1, p2) #maintain consistent exclusions w/ exceptions

        #now add the CustomBondForce for exceptions
        exception_force = self._out_system_forces['CustomExceptionForce']

        for solute_exception_term in self._solute_exceptions:
            p1, p2, [chargeProd, sigma, epsilon] = solute_exception_term
            if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
                identifier = 0
                exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])

        for interexception_term in self._interexceptions:
            p1, p2, [chargeProd, sigma, epsilon] = interexception_term
            if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
                identifier = 2
                exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])
        
        for solvent_exception_term in self._solvent_exceptions:
            p1, p2, [chargeProd, sigma, epsilon] = solvent_exception_term
            if (chargeProd.value_in_unit_system(unit.md_unit_system) != 0.0) or (epsilon.value_in_unit_system(unit.md_unit_system) != 0.0):
                identifier = 1
                exception_force.addBond(p1, p2, [chargeProd, sigma, epsilon, identifier])


In [10]:
# Compute energy for REST system
# Create REST system
res1 = list(topology.residues())[1]
rest_atoms = [atom.index for atom in res1.atoms()]
factory = REST2(system, solute_region=rest_atoms)
REST_system = factory.REST_system

# Create thermodynamic state
lambda_zero_alchemical_state = RESTState.from_system(REST_system)
thermostate = ThermodynamicState(REST_system, temperature=T_min)
compound_thermodynamic_state = CompoundThermodynamicState(thermostate,
                                                          composable_states=[lambda_zero_alchemical_state])

# Set alchemical parameters
beta_0 = 1 / (kB * T_min)
beta_m = 1 / (kB * T)
compound_thermodynamic_state.set_alchemical_parameters(beta_0, beta_m)

# Minimize and save energy
integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context = compound_thermodynamic_state.create_context(integrator)
context.setPositions(positions)
sampler_state = SamplerState.from_context(context)
REST_energy = compound_thermodynamic_state.reduced_potential(sampler_state)

# Compute energy for non-RESTified system
# Determine regions and scaling factors
solute = rest_atoms
solvent = [i for i in range(topology.getNumAtoms()) if i not in rest_atoms]
solute_scaling = beta_m / beta_0
inter_scaling = np.sqrt(beta_m / beta_0)

# Scale the terms in the bond force appropriately
bond_force = system.getForce(0)
for bond in range(bond_force.getNumBonds()):
    p1, p2, length, k = bond_force.getBondParameters(bond)
    if p1 in solute and p2 in solute:
        bond_force.setBondParameters(bond, p1, p2, length, k * solute_scaling)
    elif (p1 in solute and p2 in solvent) or (p1 in solvent and p2 in solute):
        bond_force.setBondParameters(bond, p1, p2, length, k * inter_scaling)

# Scale the terms in the angle force appropriately
angle_force = system.getForce(1)
for angle_index in range(angle_force.getNumAngles()):
    p1, p2, p3, angle, k = angle_force.getAngleParameters(angle_index)
    if p1 in solute and p2 in solute and p3 in solute:
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * solute_scaling)
    elif set([p1, p2, p3]).intersection(set(solute)) != set() and set([p1, p2, p3]).intersection(
            set(solvent)) != set():
        angle_force.setAngleParameters(angle_index, p1, p2, p3, angle, k * inter_scaling)

# Scale the terms in the torsion force appropriately
torsion_force = system.getForce(2)
for torsion_index in range(torsion_force.getNumTorsions()):
    p1, p2, p3, p4, periodicity, phase, k = torsion_force.getTorsionParameters(torsion_index)
    if p1 in solute and p2 in solute and p3 in solute and p4 in solute:
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * solute_scaling)
    elif set([p1, p2, p3, p4]).intersection(set(solute)) != set() and set([p1, p2, p3, p4]).intersection(
            set(solvent)) != set():
        torsion_force.setTorsionParameters(torsion_index, p1, p2, p3, p4, periodicity, phase, k * inter_scaling)

# Scale the exceptions in the nonbonded force appropriately
nb_force = system.getForce(3)
for nb_index in range(nb_force.getNumExceptions()):
    p1, p2, chargeProd, sigma, epsilon = nb_force.getExceptionParameters(nb_index)
    if p1 in solute and p2 in solute:
        nb_force.setExceptionParameters(nb_index, p1, p2, solute_scaling * chargeProd, sigma, solute_scaling * epsilon)
    elif (p1 in solute and p2 in solvent) or (p1 in solvent and p2 in solute):
        nb_force.setExceptionParameters(nb_index, p1, p2, inter_scaling * chargeProd, sigma, inter_scaling * epsilon)

# Scale nonbonded interactions for solute-solute region by adding exceptions for all pairs of atoms
exception_pairs = [tuple(sorted([nb_force.getExceptionParameters(nb_index)[0], nb_force.getExceptionParameters(nb_index)[1]])) for nb_index in range(nb_force.getNumExceptions())]
solute_pairs = set([tuple(sorted(pair)) for pair in list(itertools.product(solute, solute))])
for pair in list(solute_pairs):
    p1 = pair[0]
    p2 = pair[1]
    p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
    p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
    if p1 != p2:
        if pair not in exception_pairs:
            nb_force.addException(p1, p2, p1_charge * p2_charge * solute_scaling, 0.5 * (p1_sigma + p2_sigma),
                                  np.sqrt(p1_epsilon * p2_epsilon) * solute_scaling)

# Scale nonbonded interactions for inter region by adding exceptions for all pairs of atoms
for pair in list(itertools.product(solute, solvent)):
    p1 = pair[0]
    p2 = int(pair[1])  # otherwise, will be a numpy int
    p1_charge, p1_sigma, p1_epsilon = nb_force.getParticleParameters(p1)
    p2_charge, p2_sigma, p2_epsilon = nb_force.getParticleParameters(p2)
    if tuple(sorted(pair)) not in exception_pairs:
        nb_force.addException(p1, p2, p1_charge * p2_charge * inter_scaling, 0.5 * (p1_sigma + p2_sigma), np.sqrt(p1_epsilon * p2_epsilon) * inter_scaling)

# Get energy
thermostate = ThermodynamicState(system, temperature=T_min)
integrator = openmm.VerletIntegrator(1.0 * unit.femtosecond)
context = thermostate.create_context(integrator)
context.setPositions(positions)
sampler_state = SamplerState.from_context(context)
nonREST_energy = thermostate.reduced_potential(sampler_state)

INFO:REST:No MonteCarloBarostat added.
INFO:REST:getDefaultPeriodicBoxVectors added to hybrid: [Quantity(value=Vec3(x=3.2852863, y=0.0, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=3.2861648000000003, z=0.0), unit=nanometer), Quantity(value=Vec3(x=0.0, y=0.0, z=3.1855098), unit=nanometer)]
INFO:REST:No unknown forces.


In [11]:
REST_energy

-12677.453208292709

In [12]:
nonREST_energy

-9953.2906124129